# Azure Queue Storage Demo

## Preparation

In [ ]:
#r "nuget: Azure.Storage.Queues"

Import namespaces:

In [ ]:
using Azure.Storage.Queues;
using Azure.Storage.Queues.Models;

Setup connection string:

In [ ]:
var connectionString = "DefaultEndpointsProtocol=https;AccountName=ztechtest;AccountKey=XXX;EndpointSuffix=core.windows.net";

## Create and Initialize Queue Client
This step creates a queue client and initializes the queue.

In [ ]:
// Name of the queue
var queueName = "my-queue";

// Create the queue client
var queueClient = new QueueClient(connectionString, queueName);

Console.WriteLine($"Creating queue '{queueName}'...");

// Create the queue if it doesn't exist
await queueClient.CreateIfNotExistsAsync();

if (queueClient.Exists())
{
    Console.WriteLine($"Queue '{queueName}' is ready.");
}
else
{
    Console.WriteLine($"Failed to create queue '{queueName}'.");
}

## Add Messages to the Queue
Now, let's add some messages to the queue.

In [ ]:
for (int i = 1; i <= 5; i++)
{
    await queueClient.SendMessageAsync($"Message #{i}");
}

Console.WriteLine("Messages added to the queue.");

## Retrieve and process messages

In [ ]:
while (true)
{
    // Receive a message from the queue
    var msgReply = await queueClient.ReceiveMessageAsync(visibilityTimeout: TimeSpan.FromSeconds(30));

    // Check if a message was received; if not, queue is empty and exit the loop
    if (msgReply.Value == null) break;
    var msg = msgReply.Value;

    // Show message details
    msg.Display();

    // Delete the message from the queue
    await queueClient.DeleteMessageAsync(msg.MessageId, msg.PopReceipt);
}
Console.WriteLine("All messages received and deleted from the queue.");

The `visibilityTimeout` argument in `ReceiveMessageAsync` specifies how long the received message will be invisible to other consumers after being read. When a message is retrieved, it remains in the queue but is hidden for the duration of the timeout. If the message is not deleted within this period, it becomes visible again and can be processed by another consumer. This mechanism helps ensure that messages are not lost if processing fails, but also prevents multiple consumers from processing the same message simultaneously.

When message is returned into the queue and received again, its `DequeueCount` property is incremented. You may use this property to weed out _poison messages_, which can cause problems. If the same message fails processing multiple times, you can throw it away.

First, let's add some messages again. They will contain numbers, but some of them are invalid and would crash the processing code:

In [ ]:
Console.Write("Generating: ");
for (int i = 1; i <= 20; i++) {
    var randomNumber = new Random().Next(1, 101);

    if (randomNumber <= 10) {
        // In 10% of the cases, intentionally generate invalid message 
        await queueClient.SendMessageAsync("Poison!");
        Console.Write("!");
    } else {
        await queueClient.SendMessageAsync(randomNumber.ToString());
        Console.Write(".");
    }
}

Now we will try to process the messages, but some will fail:

In [ ]:
const int MaxRetry = 3;
const int MaxWaitTime = 10;

var sw = new System.Diagnostics.Stopwatch();

while(true) {
    // Receive a message from the queue
    var msgReply = await queueClient.ReceiveMessageAsync(visibilityTimeout: TimeSpan.FromSeconds(1));

    // Check if a message was received; if not, wait for a while
    if (msgReply.Value == null) {
        if(sw.Elapsed > TimeSpan.FromSeconds(MaxWaitTime)) {
            sw.Stop();
            Console.WriteLine("No messages in the queue, exiting.");
            break;
        }
        sw.Start();
        await Task.Delay(TimeSpan.FromSeconds(1));
        continue;
    }
    var msg = msgReply.Value;
    sw.Reset();

    // Check if the message wasn't already tried enough times
    if (msg.DequeueCount > MaxRetry) {
        // This is poison message, delete it
        Console.WriteLine($"Message #{msg.MessageId}: '{msg.Body}' is a poison message.");
        await queueClient.DeleteMessageAsync(msg.MessageId, msg.PopReceipt);
    } else {
        // Process the message
        try {
            Console.Write($"Processing message #{msg.MessageId}: '{msg.Body}'...");
            var x = int.Parse(msg.Body.ToString());
            await queueClient.DeleteMessageAsync(msg.MessageId, msg.PopReceipt);
            Console.WriteLine("OK");
        } catch (Exception ex) {
            Console.WriteLine($"Failed: {ex.Message}");
        }
    }
}

## Clean-up

In [ ]:
await queueClient.DeleteIfExistsAsync();
Console.WriteLine($"Queue '{queueName}' deleted.");

Note: As with blob containers, if you delete a queue, it's just marked for deletion and it takes a while to be actually deleted. If you will try to recreate the queue immediatelly, it will fail.